<a href="https://colab.research.google.com/github/SILVIAIRENE/Data-Scientist-Machine-Learning-Engineer-Introductory-Course/blob/master/ResNet_y_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sprint 20: Mejora de Precisión de Segmentación con Transfer Learning

## TGS Salt Identification Challenge - Comparación ResNet vs VGG
## ✅ VERSIÓN ACTUALIZADA 2025 - TENSORFLOW 2.x

'Este notebook implementa y compara dos arquitecturas de U-Net usando transfer learning:'
#- **U-Net con encoder ResNet50**
#- **U-Net con encoder VGG16**

### Objetivos:
#1. **Problema 1**: Revisar el código con transfer learning
#2. **Problema 2**: Cambiar de ResNet a VGG en el encoder
#3. **Problema 3**: Entrenar ambos modelos y comparar resultados

### Versiones Actualizadas:
#- TensorFlow 2.19+ (con Keras integrado)
#- Python 3.12
#- Dependencias compatibles con 2025'''
print("=== SPRINT 20: SEGMENTACIÓN CON TRANSFER LEARNING ===")
print("TGS Salt Identification Challenge")
print("Comparación ResNet50-UNet vs VGG16-UNet")
print("🚀 VERSIÓN ACTUALIZADA 2025 - TENSORFLOW 2.x")

# Verificar versiones
import sys
print(f"Python version: {sys.version}")

# Instalar/actualizar dependencias compatibles
!pip install --upgrade tensorflow>=2.19.0
!pip install --upgrade scikit-image
!pip install --upgrade matplotlib
!pip install --upgrade opencv-python
!pip install --upgrade seaborn
!pip install --upgrade kaggle

print("✅ Dependencias instaladas correctamente")
# Importar librerías necesarias
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import cv2
from skimage.transform import resize
from skimage import exposure
import warnings
warnings.filterwarnings('ignore')

# TensorFlow 2.x con Keras integrado
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate,
    BatchNormalization, Activation, Dropout, GlobalAveragePooling2D,
    Dense, Lambda, Conv2DTranspose
)
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# Configurar seeds para reproducibilidad (sintaxis TF 2.x)
np.random.seed(42)
tf.random.set_seed(42)

# Configurar matplotlib
plt.style.use('default')  # Usar estilo por defecto más compatible
%matplotlib inline

print("✅ Todas las importaciones exitosas")
# Configuración de memoria GPU para evitar errores de memoria
try:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU configurada: {len(gpus)} dispositivos encontrados")
    else:
        print("⚠️  No se encontraron GPUs, usando CPU")
except:
    print("⚠️  Configuración de GPU omitida")

# Verificar dispositivos disponibles
print("Dispositivos disponibles:")
print(tf.config.list_physical_devices())
print("=== DESCARGA DE DATOS TGS SALT IDENTIFICATION CHALLENGE ===")

# Configurar Kaggle API
import os
from pathlib import Path

# Verificar si kaggle.json existe
kaggle_path = Path.home() / '.kaggle' / 'kaggle.json'
if not kaggle_path.exists():
    print("❌ Por favor, sube tu archivo kaggle.json primero")
    print("1. Ve a Account > Create New API Token en Kaggle")
    print("2. Sube el archivo kaggle.json usando el botón de archivos de Colab")
    print("3. Ejecuta esta celda nuevamente")
else:
    print("✅ Archivo kaggle.json encontrado")

# Crear directorio y configurar permisos
!mkdir -p ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

try:
    # Descargar dataset
    !kaggle competitions download -c tgs-salt-identification-challenge
    !unzip -q tgs-salt-identification-challenge.zip -d ./tgs-data/

    # Listar archivos descargados
    !ls -la ./tgs-data/
    print("✅ Dataset descargado exitosamente")
except Exception as e:
    print(f"❌ Error descargando dataset: {e}")
    print("Verifica que hayas aceptado las reglas de la competencia en Kaggle")
    # Parámetros del modelo actualizados
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
BATCH_SIZE = 16
EPOCHS = 30  # Reducido para pruebas más rápidas
LEARNING_RATE = 1e-4
VALIDATION_SPLIT = 0.2

# Rutas de datos actualizadas
DATA_PATH = './tgs-data/'
TRAIN_PATH = os.path.join(DATA_PATH, 'train/')
TEST_PATH = os.path.join(DATA_PATH, 'test/')

print(f"📋 CONFIGURACIÓN:")
print(f"- Tamaño de imagen: {IMG_WIDTH}x{IMG_HEIGHT}x{IMG_CHANNELS}")
print(f"- Batch size: {BATCH_SIZE}")
print(f"- Épocas: {EPOCHS}")
print(f"- Learning rate: {LEARNING_RATE}")
print(f"- Ruta de datos: {DATA_PATH}")

# Verificar que los directorios existen
if os.path.exists(TRAIN_PATH):
    print("✅ Directorio de entrenamiento encontrado")
else:
    print("❌ Directorio de entrenamiento no encontrado")
def rle_decode(mask_rle, shape=(101, 101)):
    """
    Decodifica Run Length Encoding a máscara binaria
    """
    if pd.isna(mask_rle):
        return np.zeros(shape, dtype=np.uint8)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

def rle_encode(img):
    """
    Codifica máscara binaria a Run Length Encoding
    """
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def load_and_preprocess_data():
    """
    Carga y preprocesa los datos de entrenamiento
    """
    try:
        # Cargar metadatos
        train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
        depths_df = pd.read_csv(os.path.join(DATA_PATH, 'depths.csv'))

        # Merge con información de profundidad
        train_df = train_df.merge(depths_df, on='id', how='left')

        print(f"📊 ESTADÍSTICAS DEL DATASET:")
        print(f"- Total de imágenes de entrenamiento: {len(train_df)}")
        print(f"- Imágenes con máscaras: {len(train_df[~train_df.rle_mask.isna()])}")
        print(f"- Imágenes sin máscaras: {len(train_df[train_df.rle_mask.isna()])}")

        return train_df
    except Exception as e:
        print(f"❌ Error cargando datos: {e}")
        return None

def create_data_generators(train_df, validation_split=0.2):
    """
    Crea generadores de datos para entrenamiento y validación
    """
    if train_df is None:
        return None, None

    # Dividir en entrenamiento y validación
    train_ids, val_ids = train_test_split(
        train_df.id.values,
        test_size=validation_split,
        stratify=train_df.rle_mask.isna(),
        random_state=42
    )

    print(f"📈 DIVISIÓN DE DATOS:")
    print(f"- Imágenes de entrenamiento: {len(train_ids)}")
    print(f"- Imágenes de validación: {len(val_ids)}")

    return train_ids, val_ids

def load_image(image_id, is_train=True):
    """
    Carga y preprocesa una imagen individual
    """
    try:
        path = TRAIN_PATH if is_train else TEST_PATH

        # Cargar imagen
        img_path = os.path.join(path, 'images', f'{image_id}.png')
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            raise ValueError(f"No se pudo cargar la imagen: {img_path}")

        # Convertir a RGB duplicando el canal
        img = np.stack([img, img, img], axis=-1)

        # Redimensionar
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), preserve_range=True)

        return img.astype(np.float32) / 255.0
    except Exception as e:
        print(f"Error cargando imagen {image_id}: {e}")
        return np.zeros((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.float32)

def load_mask(image_id, train_df):
    """
    Carga y preprocesa una máscara individual
    """
    try:
        rle_mask = train_df[train_df.id == image_id].rle_mask.iloc[0]
        mask = rle_decode(rle_mask)
        mask = resize(mask, (IMG_HEIGHT, IMG_WIDTH), preserve_range=True)
        return mask.astype(np.float32)
    except Exception as e:
        print(f"Error cargando máscara {image_id}: {e}")
        return np.zeros((IMG_HEIGHT, IMG_WIDTH), dtype=np.float32)

# Cargar datos
print("🔄 Cargando datos...")
train_df = load_and_preprocess_data()

if train_df is not None:
    train_ids, val_ids = create_data_generators(train_df, VALIDATION_SPLIT)
    print("✅ Datos cargados exitosamente")
else:
    print("❌ Error en la carga de datos")
    class DataGenerator(tf.keras.utils.Sequence):
    """
    Generador de datos personalizado compatible con TensorFlow 2.x
    """
    def __init__(self, image_ids, train_df, batch_size=16, shuffle=True):
        self.image_ids = image_ids
        self.train_df = train_df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_ids) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_ids = [self.image_ids[k] for k in indexes]
        X, y = self.__data_generation(batch_ids)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_ids))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_ids):
        X = np.empty((self.batch_size, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
        y = np.empty((self.batch_size, IMG_HEIGHT, IMG_WIDTH, 1))

        for i, image_id in enumerate(batch_ids):
            # Cargar imagen
            X[i] = load_image(image_id)

            # Cargar máscara
            mask = load_mask(image_id, self.train_df)
            y[i] = np.expand_dims(mask, axis=-1)

        return X, y

if train_ids is not None and val_ids is not None:
    # Crear generadores
    train_generator = DataGenerator(train_ids, train_df, BATCH_SIZE, shuffle=True)
    val_generator = DataGenerator(val_ids, train_df, BATCH_SIZE, shuffle=False)

    print(f"📊 GENERADORES CREADOS:")
    print(f"- Generador de entrenamiento: {len(train_generator)} batches")
    print(f"- Generador de validación: {len(val_generator)} batches")
    print("✅ Generadores listos")
else:
    print("❌ No se pudieron crear los generadores")
    def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """
    Calcula el coeficiente Dice (F1-score para segmentación)
    Compatible con TensorFlow 2.x
    """
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    """
    Función de pérdida basada en Dice coefficient
    """
    return 1 - dice_coefficient(y_true, y_pred)

def combined_loss(y_true, y_pred):
    """
    Combina Binary Cross-Entropy con Dice Loss
    """
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return 0.5 * bce + 0.5 * dice

def iou_metric(y_true, y_pred, smooth=1e-6):
    """
    Calcula Intersection over Union (IoU)
    Compatible con TensorFlow 2.x
    """
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

# Función de Tversky Loss (adicional para mejor rendimiento)
def tversky_loss(y_true, y_pred, alpha=0.3, beta=0.7, smooth=1e-6):
    """
    Tversky Loss - útil para datos desbalanceados
    """
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)

    true_pos = tf.reduce_sum(y_true_f * y_pred_f)
    false_neg = tf.reduce_sum(y_true_f * (1 - y_pred_f))
    false_pos = tf.reduce_sum((1 - y_true_f) * y_pred_f)

    tversky = (true_pos + smooth) / (true_pos + alpha * false_neg + beta * false_pos + smooth)
    return 1 - tversky

print("✅ Métricas definidas: Dice Coefficient, Dice Loss, Combined Loss, IoU, Tversky Loss")
print("=" * 70)
print("PROBLEMA 1: ANÁLISIS DE TRANSFER LEARNING")
print("=" * 70)

print("""
🎯 ¿QUÉ ES TRANSFER LEARNING?

Transfer Learning es una técnica donde utilizamos un modelo pre-entrenado
en un dataset grande (como ImageNet) y adaptamos sus características
aprendidas para nuestro problema específico.

🔄 DIFERENCIAS CON LA IMPLEMENTACIÓN BÁSICA:

1. **Encoder Pre-entrenado**: En lugar de entrenar desde cero, utilizamos
   pesos de modelos entrenados en ImageNet (millones de imágenes)

2. **Feature Extraction**: Aprovechamos características de bajo nivel ya
   aprendidas (bordes, texturas, formas, patrones)

3. **Convergencia más rápida**: El modelo converge más rápido al partir
   de pesos pre-entrenados

4. **Mejor precisión**: Especialmente útil cuando tenemos datasets limitados

5. **Menos recursos**: Requiere menos tiempo de entrenamiento

⚙️ CÓMO IMPLEMENTAMOS TRANSFER LEARNING:

1. **Cargar modelo pre-entrenado**: ResNet50 o VGG16 entrenados en ImageNet
2. **Congelar capas iniciales**: Para mantener características de bajo nivel
3. **Fine-tuning**: Permitir el entrenamiento de capas superiores
4. **Adaptación de decoder**: Diseñar decoder específico para segmentación
5. **Skip connections**: Conectar capas del encoder con el decoder

📊 VENTAJAS EN SEGMENTACIÓN:
- Mejor detección de bordes y contornos
- Reconocimiento de texturas complejas
- Adaptación rápida a dominios específicos
- Menor sobreajuste con datos limitados
""")
def build_resnet_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    """
    Construye U-Net con encoder ResNet50 pre-entrenado
    Compatible con TensorFlow 2.x y Keras integrado
    """

    # =============================================
    # ENCODER: ResNet50 Pre-entrenado
    # =============================================

    # Cargar ResNet50 pre-entrenado (sin clasificador final)
    base_model = ResNet50(
        weights='imagenet',  # Pesos pre-entrenados en ImageNet
        include_top=False,   # Sin clasificador final
        input_shape=input_shape,
        input_tensor=None
    )

    print(f"🔥 ResNet50 cargado: {len(base_model.layers)} capas")

    # Obtener capas específicas para skip connections
    # Estas capas proporcionan características a diferentes resoluciones
    layer_names = [layer.name for layer in base_model.layers]
    print(f"📋 Capas disponibles: {len(layer_names)} capas")

    # Intentar encontrar las capas apropiadas
    skip_layers = []
    skip_outputs = []

    # Buscar capas por patrones conocidos
    input_layer = base_model.input
    skip_outputs.append(input_layer)  # Input original (128x128)

    # Buscar capas intermedias
    for layer in base_model.layers:
        if any(pattern in layer.name.lower() for pattern in
               ['conv1', 'conv2_block3', 'conv3_block4', 'conv4_block6']):
            if hasattr(layer, 'output'):
                skip_outputs.append(layer.output)
                skip_layers.append(layer.name)

    print(f"🔗 Skip connections encontradas: {len(skip_outputs)}")
    for i, name in enumerate(skip_layers):
        print(f"   - Capa {i+1}: {name}")

    # Bottleneck (centro de la U-Net)
    bottleneck = base_model.output
    print(f"🏗️ Bottleneck shape: {bottleneck.shape}")

    # =============================================
    # DECODER: Upsampling con Skip Connections
    # =============================================

    # Decoder Block 1: Upsampling inicial
    x = UpSampling2D(size=(2, 2))(bottleneck)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Decoder Block 2
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Skip connection si está disponible
    if len(skip_outputs) > 4:
        try:
            x = concatenate([x, skip_outputs[-1]])
        except:
            pass  # Continuar sin skip connection si hay error

    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Decoder Block 3
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Skip connection si está disponible
    if len(skip_outputs) > 3:
        try:
            # Ajustar dimensiones si es necesario
            skip_tensor = skip_outputs[-2]
            if skip_tensor.shape[1:3] == x.shape[1:3]:
                x = concatenate([x, skip_tensor])
        except:
            pass

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Decoder Block 4
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Skip connection con input original
    if len(skip_outputs) > 0:
        try:
            if skip_outputs[0].shape[1:3] == x.shape[1:3]:
                x = concatenate([x, skip_outputs[0]])
        except:
            pass

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Final upsampling si es necesario
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # =============================================
    # OUTPUT LAYER
    # =============================================

    # Capa final de segmentación
    output = Conv2D(1, (1, 1), activation='sigmoid', name='segmentation_output')(x)

    # Crear modelo
    model = Model(inputs=base_model.input, outputs=output, name='ResNet50_UNet')

    # =============================================
    # CONFIGURACIÓN DE TRANSFER LEARNING
    # =============================================

    # Congelar capas del encoder (transfer learning)
    num_layers_to_freeze = len(base_model.layers) - 20  # Congelar todas menos las últimas 20

    for i, layer in enumerate(base_model.layers):
        if i < num_layers_to_freeze:
            layer.trainable = False
        else:
            layer.trainable = True

    trainable_layers = sum([1 for layer in model.layers if layer.trainable])
    frozen_layers = len(model.layers) - trainable_layers

    print(f"🏗️ MODELO RESNET-UNET CREADO:")
    print(f"   - Capas totales: {len(model.layers)}")
    print(f"   - Capas entrenables: {trainable_layers}")
    print(f"   - Capas congeladas: {frozen_layers}")
    print(f"   - Parámetros totales: {model.count_params():,}")

    return model

# Crear modelo ResNet-UNet
print("=" * 70)
print("🏗️ CONSTRUYENDO MODELO U-NET CON RESNET50")
print("=" * 70)

try:
    resnet_model = build_resnet_unet()

    # Compilar modelo con optimizador moderno
    resnet_model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),  # Sintaxis actualizada
        loss=combined_loss,
        metrics=[dice_coefficient, iou_metric, 'binary_accuracy']
    )

    print("✅ Modelo ResNet-UNet compilado exitosamente")

    # Mostrar resumen del modelo (opcional)
    # resnet_model.summary()

except Exception as e:
    print(f"❌ Error creando modelo ResNet: {e}")
    resnet_model = None
    print("=" * 70)
print("PROBLEMA 2: REESCRITURA DE CÓDIGO - RESNET A VGG")
print("=" * 70)

def build_vgg_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    """
    Construye U-Net con encoder VGG16 pre-entrenado
    Compatible con TensorFlow 2.x

    DIFERENCIAS CON RESNET:
    1. VGG16 tiene arquitectura más simple (secuencial)
    2. No tiene skip connections residuales internas
    3. Usa solo convoluciones 3x3 y max pooling
    4. Menos parámetros que ResNet50
    5. Arquitectura más interpretable
    """

    # =============================================
    # ENCODER: VGG16 Pre-entrenado
    # =============================================

    # Cargar VGG16 pre-entrenado (sin clasificador final)
    base_model = VGG16(
        weights='imagenet',  # Pesos pre-entrenados en ImageNet
        include_top=False,   # Sin clasificador final
        input_shape=input_shape,
        input_tensor=None
    )

    print(f"🔥 VGG16 cargado: {len(base_model.layers)} capas")

    # VGG16 tiene nombres de capas más predecibles
    skip_connections = {}

    # Buscar capas específicas para skip connections
    for layer in base_model.layers:
        if 'block1_conv2' in layer.name:
            skip_connections['block1'] = layer.output
        elif 'block2_conv2' in layer.name:
            skip_connections['block2'] = layer.output
        elif 'block3_conv3' in layer.name:
            skip_connections['block3'] = layer.output
        elif 'block4_conv3' in layer.name:
            skip_connections['block4'] = layer.output

    print(f"🔗 Skip connections VGG16: {len(skip_connections)} encontradas")
    for name, tensor in skip_connections.items():
        print(f"   - {name}: {tensor.shape}")

    # Input y Bottleneck
    input_tensor = base_model.input
    bottleneck = base_model.output  # 4x4x512
    print(f"🏗️ Bottleneck VGG16 shape: {bottleneck.shape}")

    # =============================================
    # DECODER: Upsampling con Skip Connections
    # =============================================

    # Decoder Block 1: 4x4 -> 8x8
    x = UpSampling2D(size=(2, 2))(bottleneck)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)


"..."

"Let me reevaluate and take a different approach."

IndentationError: expected an indented block after class definition on line 265 (ipython-input-3446721042.py, line 266)